In [1]:
print(f"** curator script **")

** curator script **


In [2]:
from elasticsearch import Elasticsearch
import os
import json
import pandas as pd
import jsondiff
import logging
from dotenv import load_dotenv
import dotenv
import socket
import requests
import base64
import sys
import logging
from requests.auth import HTTPBasicAuth
from IPython.display import HTML
from cryptography import x509
import datetime
from cryptography.hazmat.backends import default_backend
import warnings
warnings.filterwarnings("ignore")

In [3]:
''' pip install python-dotenv'''
# load_dotenv() # will search for .env file in local folder and load variables
# Reload the variables from your .env file, overriding existing ones
dotenv.load_dotenv(".env", override=True)

False

In [4]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

In [5]:
logger = logging.getLogger("api-call")

In [6]:
''' base64 encode '''
def base64_encode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    encoded = '{}'.format(base64.b64encode(id_pass.encode('utf-8')).decode())
    # print(encoded)
    return encoded

In [7]:
''' base64 decode '''
def base64_decode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    return base64.b64decode(id_pass).decode('utf-8')

In [8]:
def get_headers(auth):
    ''' Elasticsearch Header '''
    ''' 
    Basic Authentication is a method for an HTTP user agent (e.g., a web browser) 
    to provide a username and password when making a request. 
    You can send the authorization header 
    when making requests and accessing to ES Cluster based on Search-Guard as X-pack. 
      
    Basic Auth : 
    {
        'Content-type': 'application/json', 
        'Authorization' : 'Basic base64.encode(id:password), 
        'Connection': 'close'
    }
    '''
    base64_encoded_user_credential = os.getenv(auth)
    # base64_encoded_user_credential = auth
    # print(f"base64_encoded_user_credential : {base64_encoded_user_credential}")
    # print(f"User - {base64_decode_for_search_guard(base64_encoded_user_credential)}")
    headers = {
            'Content-type': 'application/json', 
            'Authorization' : 'Basic {}'.format(base64_encoded_user_credential),
            'Connection': 'close'
    }
    # print(f"headers : {json.dumps(headers, indent=2)}")

    return headers

In [9]:
''' dev '''
source_es_host = "http://localhost:9201"

In [10]:
def get_es_instance(auth):
    """ 
    Create an instance of the secure ES cluster by passing multiple ES hosts, a CA certificate file, and a username/password for basic auth as parameters
    """
    try:
        print(f'source_es_host : {source_es_host}')
        es_client = Elasticsearch(
                                  hosts="{}".format(source_es_host), 
                                  headers=get_headers(auth), 
                                  # http_auth=('es_admin', 'test'),
                                  timeout=5,  
                                  verify_certs=False
                                 )
        print(f"An instance of the ES client : {es_client}")
        return es_client
    except Exception as e:
        print(str(e))

In [11]:
auth = ""

In [12]:
''' base64 encodede(username:password) from .env file '''
es_client = get_es_instance(auth)

source_es_host : http://localhost:9201
An instance of the ES client : <Elasticsearch([{'host': 'localhost', 'port': 9201}])>


In [13]:
# index_pattern="logstash*"
index_pattern="test*"

In [14]:
try:
    print("** The health of the secure ES cluster **")
    print(json.dumps(es_client.cat.health(format="json"), indent=2))
    # print(json.dumps(es_client.cat.indices(params={"format": "json"}), indent=2))
    # es_health_dict = es_client.cluster.health()
    # es_health_dict = es_client_wx.indices.get("*")
    es_indices_list = list(es_client.indices.get(index_pattern).keys())
    # print(es_indices_list)
    # print("** A list of ES Indices for wxusers **")
    # print(json.dumps(es_indices_list, indent=2))
    # df = pd.DataFrame.from_dict([es_wxusers_es_indices_list])
    # display(df)
except Exception as e:
    print(str(e))

** The health of the secure ES cluster **
2025-10-17 10:38:59,257 - elasticsearch - INFO - GET http://localhost:9201/_cat/health?format=json [status:200 request:0.055s]
[
  {
    "epoch": "1760715539",
    "timestamp": "15:38:59",
    "cluster": "docker-cluster",
    "status": "yellow",
    "node.total": "1",
    "node.data": "1",
    "shards": "339",
    "pri": "339",
    "relo": "0",
    "init": "0",
    "unassign": "339",
    "pending_tasks": "0",
    "max_task_wait_time": "-",
    "active_shards_percent": "50.0%"
  }
]
2025-10-17 10:38:59,335 - elasticsearch - INFO - GET http://localhost:9201/test* [status:200 request:0.075s]


In [15]:
df = pd.DataFrame.from_dict({'ES Indices' : es_indices_list})
display(df)

,ES Indices
0,test
1,test-index
2,test-index1
